In [ ]:
import os
os.chdir("..")
from contentcreatie.llm_client.prompt_builder import  PromptBuilder
import pandas as pd
from contentcreatie.config.settings import Settings
from contentcreatie.config.paths import paths
from test_notebooks.content_retrieval import llm,doc_store, vector_store
from pipelines.processors import summarize_new_documents, add_new_documents_to_docstore
import sys
settings = Settings()


In [ ]:
kme_vertaaltabel = pd.read_csv(paths.base_dir /"data" /  "kme_vertaaltabel.csv",sep=';').set_index("KME_ID")
kme_vertaaltabel['BELASTINGSOORT'] = kme_vertaaltabel['BELASTINGSOORT'].str.split('/',n=1).str[0]

# Opbouwen van de Document Store

In [ ]:
kme_documents = pd.read_pickle(paths.content_folder / "extracted_df_km.pkl",compression='gzip').reset_index()
kme_documents = kme_documents.set_index("km_nummer").join(kme_vertaaltabel).reset_index()

In [ ]:
results = add_new_documents_to_docstore(kme_documents,doc_store)

In [ ]:
current_docs = kme_documents.km_nummer.values
current_keys = list(doc_store.documents.keys())
for kmnr in current_keys:
    if kmnr not in current_docs:
        doc_store.documents.pop(kmnr)

### Samenvatten en segmenteren content

In [ ]:
summary_processor = PromptBuilder(template_path=paths.base_dir/'Contentcreatie'/ 'prompt_templates',name='summarize')

In [ ]:
import logging
import sys

# Set up logging to see all debug messages from the Azure SDK
logger = logging.getLogger('azure')
logger.setLevel(logging.DEBUG)

# Create a handler to write logs to standard out
handler = logging.StreamHandler(stream=sys.stdout)
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

In [ ]:
stats = summarize_new_documents(
    doc_store=doc_store,
    prompt_builder=summary_processor,
    llm=llm,
    max_workers=20,
    start=0,
    count=7000,
    show_progress=True
)

In [ ]:
doc_store.rebuild_search_index()
doc_store.save()

### Update taxonomie

In [ ]:
vector_store.sync_with_store()